# DSCI 100 Final Project - Group 05 - [Title]
**Group Members:** Caitlyn Woods, .... 


## Introduction

Introduction Expectations (will be deleted before submission)
- provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report
- clearly state the question you tried to answer with your project
- identify and fully describe the dataset that was used to answer the question

Section 1: What are we doing and how (breaking down the data science for someone unfamiliar)
Section 2: What broad and specific questions are we answering? Why?
Section 3: Describe the dataset we are going to use. 

In this report, we will analyze data collected by a UBC Computer Science research group using strategies taught in the DSCI 100: Introduction to Data Science course to answer a research question we have formulated. However, before it is possible to discuss the specifics of the research question and datasets, it is crucial to have a basic understanding of the tools and strategies used throughout this report. Simply put, we will be using a variety of strategies, including summarizing, visualizing, and modelling, to gain a better understanding of and derive useful information from the data we have been provided. The specifics of these strategies will be explained in the "methods and results" section. All code included in this report will be written in R, and will use functions from several libraries, notably the Tidyverse and ggplot2 libraries [If there are any others that need to be mentioned, let me know or add them here]. When we refer to a "dataset", we are referencing a specific table of data, while an "observation" refers to a row of this table, and a "variable" is a column of the table. 

In this report, we will attempt to answer the broad question: "